In [2]:
!pip install yfinance
!pip install ta
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib
import ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=e4ee69c558c7718db3de9826ab056a6feea2de421c3c3358a575f8b696318c5a
  Stored in directory: /Users/anshulkedia/Library/Caches/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [3]:
# # Download stock data
# TICKER = 'RELIANCE.NS'
# data = yf.download(TICKER, start='2015-01-01', end='2024-01-01')
# close_prices = data[['Close']]

/var/folders/d9/qpj9c7z11zl2_410n7td44w40000gn/T/ipykernel_4850/4011050199.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(TICKER, start='2015-01-01', end='2024-01-01')
[*********************100%***********************]  1 of 1 completed


In [4]:
# # Preprocessing
# scaler = MinMaxScaler()
# scaled_data = scaler.fit_transform(close_prices)

In [5]:
# X, y = [], []
# n_input = 60
# n_output = 7
# for i in range(n_input, len(scaled_data) - n_output):
#     X.append(scaled_data[i - n_input:i])
#     y.append(scaled_data[i:i + n_output])

# X, y = np.array(X), np.array(y)

In [6]:
# # Build LSTM model
# model = Sequential()
# model.add(LSTM(100, return_sequences=True, input_shape=(X.shape[1], 1)))
# model.add(LSTM(100))
# model.add(Dense(n_output))

# model.compile(optimizer='adam', loss='mean_squared_error')
# model.fit(X, y, epochs=20, batch_size=32)

Epoch 1/20


/Users/anshulkedia/miniconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0731
Epoch 2/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0015
Epoch 3/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0013
Epoch 4/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0012
Epoch 5/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0013
Epoch 6/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0012
Epoch 7/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0012
Epoch 8/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 9.5476e-04
Epoch 9/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0012
Epoch 10/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0013
Epoch 11/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 9.9081e-04
Epoch 12/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 9.2503e-04
Epoch 13/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 8.7108e-04
Epoch 14/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 9.3124e-04
Epoch 15/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 8.

In [8]:
# # Save model and scaler
# model.save('lstm_7day_model.keras')
# joblib.dump(scaler, 'scaler.save')

['scaler.save']

In [5]:
 
# 1. Download historical stock data
TICKER = 'RELIANCE.NS'
data = yf.download(TICKER, start='2015-01-01', end='2024-01-01')

# 2. Add technical indicators

from ta.trend import SMAIndicator, MACD
from ta.momentum import RSIIndicator


sma = SMAIndicator(close=data['Close'], window=20)
rsi = RSIIndicator(close=data['Close'], window=14)
macd = MACD(close=data['Close'])

data['SMA_20'] = sma.sma_indicator()
data['RSI_14'] = rsi.rsi()
data['MACD'] = macd.macd_diff()

# 3. Drop missing values
data = data.dropna()

# 4. Define features
features = data[['Close', 'SMA_20', 'RSI_14', 'MACD', 'Volume']]
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(features)

# 5. Create sequences
X, y = [], []
n_input = 60
n_output = 7
for i in range(n_input, len(scaled_data) - n_output):
    X.append(scaled_data[i - n_input:i])        # shape: (60, 5)
    y.append(scaled_data[i:i + n_output, 0])    # Predicting 7-day Close price only

X, y = np.array(X), np.array(y)

# 6. Build LSTM model with multi-feature input
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(LSTM(100))
model.add(Dense(n_output))

model.compile(optimizer='adam', loss='mean_squared_error')

# 7. Train the model
model.fit(X, y, epochs=20, batch_size=32)

# 8. Save model and scaler
model.save('lstm_7day_model.keras')
joblib.dump(scaler, 'scaler.save')

print("✅ Model trained and saved with technical indicators.")

/var/folders/d9/qpj9c7z11zl2_410n7td44w40000gn/T/ipykernel_81001/2409743887.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(TICKER, start='2015-01-01', end='2024-01-01')
[*********************100%***********************]  1 of 1 completed


ValueError: Data must be 1-dimensional, got ndarray of shape (2221, 1) instead